<a href="https://colab.research.google.com/github/kumudharam/Transformers/blob/main/TCG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install timeline 
!pip install pipeline
!pip install pipelines 
!pip install openpyxl 
!pip install xlrd
!pip install xlsxwriter 

     |████████████████████████████████| 2.9 MB 5.2 MB/s 
     |████████████████████████████████| 56 kB 4.3 MB/s 
     |████████████████████████████████| 895 kB 46.1 MB/s 
     |████████████████████████████████| 3.3 MB 37.1 MB/s 
     |████████████████████████████████| 596 kB 49.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Created wheel for timeline: filename=timeline-0.0.7-py3-none-any.whl size=17005 sha256=b92695102d5c8fc5a27a96d526121c9cc8c0e72f5d102a4d693a824ace6798f5
  Stored in directory: /root/.cache/pip/wheels/eb/03/7d/af6f2e055c7a95e6e9dcd5e64a9738678cac6bd8b05d4e1bd3
Successfully built timeline
     |████████████████████████████████| 161 kB 5.2 MB/s 
     |████████████████████████████████| 161 kB 40.3 MB/s 
     |████████████████████████████████| 161 kB 39.6 MB/s 
     |████████████████████████████████| 159 kB 41.9 MB/s 
     |████████████████████████████████| 159 

In [ ]:
!pip install pandas nltk spacy

In [ ]:
import nltk

# Download Punkt Sentence Tokenizer
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import os

# On Jupyter Notebook or Colab
DIR_PATH = os.getcwd()
# On Python module
#DIR_PATH = os.path.dirname(__file__)

FILE_PATH = os.path.join(DIR_PATH, "scripts.csv")

In [ ]:
import pandas as pd

# Read the file
df = pd.read_csv(r'/content/TestData.csv')
# Remove NaN values
df = df[~df["Dialogue"].isna()]
# Assign first_dialogue to the first row's "Dialogue" column
first_dialogue = df.loc[0, "Dialogue"]
print(first_dialogue)

GIVEN that I'm on any Screen of the application.


In [ ]:
df.head()

,Dialogue
0,GIVEN that I'm on any Screen of the application.
1,WHEN I fill the created before as like other f...
2,AND click the Apply filters to get the filtere...
3,THEN the application will display the data bas...
4,IF I close the application and Launch again.


In [ ]:
# use Python's split
first_dialogue.split(".")

["GIVEN that I'm on any Screen of the application", '']

In [ ]:
from nltk.tokenize import sent_tokenize

sent_tokenize(first_dialogue)

["GIVEN that I'm on any Screen of the application."]

In [ ]:
%%timeit -n 10
df.loc[:5000, "Dialogue"].apply(lambda x: sent_tokenize(x))

10 loops, best of 5: 584 µs per loop


In [ ]:
import spacy
# use spacy with the dependency parse 
nlp = spacy.load("en_core_web_sm")
[str(sent) for sent in nlp(first_dialogue).sents]

["GIVEN that I'm on any Screen of the application."]

In [ ]:
#%%timeit -n 10
# WARNING: takes a long time!
#df.loc[:5000, "Dialogue"].apply(lambda x: [sent.text for sent in nlp(x).sents])

In [ ]:
from spacy.lang.en import English
# use spacy with the sentencizer
nlp = English()  # just the language with no model
sentencizer = nlp.create_pipe("sentencizer")
nlp.add_pipe(sentencizer)
[str(sent) for sent in nlp(first_dialogue).sents]

["GIVEN that I'm on any Screen of the application."]

In [ ]:
%%timeit -n 10
df.loc[:5000, "Dialogue"].apply(lambda x: [sent.text for sent in nlp(x).sents])

10 loops, best of 5: 758 µs per loop


In [ ]:
import re
# use regular expression
rule = r"(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s"
re.split(rule, first_dialogue)

["GIVEN that I'm on any Screen of the application."]

In [ ]:
%%timeit -n 10
df.loc[:5000, "Dialogue"].apply(lambda x: re.split(rule, x))

10 loops, best of 5: 361 µs per loop


In [ ]:
%%timeit -n 10
# Without dependency parser
tokenized_data = []
first_5000_rows = df.loc[:5000, "Dialogue"]
for doc in nlp.pipe(first_5000_rows, batch_size=20):
  tokenized_data.append([sent.text for sent in doc.sents])

10 loops, best of 5: 470 µs per loop


In [ ]:
# Transform data using spaCy
nlp = spacy.load("en_core_web_sm")
# WARNING: takes a long time!
df["Dialogue"] = df["Dialogue"].apply(lambda x: [sent.text for sent in nlp(x).sents])

In [ ]:
df = df.explode("Dialogue", ignore_index=True)

In [ ]:
df.rename(columns={"Unnamed: 0": "Dialogue ID"}, inplace=True)
df.index.name = "Sentence ID"

In [ ]:
df.head()

,Dialogue
Sentence ID,
0,GIVEN that I'm on any Screen of the application.
1,WHEN I fill the created before as like other f...
2,AND click the Apply filters to get the filtere...
3,THEN the application will display the data bas...
4,IF I close the application and Launch again.


In [ ]:
Text = df.to_csv("scripts_tokenized.csv")
print(df.to_csv("scripts_tokenized.csv"))

In [ ]:
!pip install -U transformers==3.0.0 --quiet
!python -m nltk.downloader punkt

!git clone https://github.com/patil-suraj/question_generation.git
%cd question_generation
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import word_tokenize, sent_tokenize
from keras.preprocessing.text import Tokenizer

from transformers import pipeline
import timeline
import pandas as pd
import csv 
import openpyxl
import xlsxwriter
from itertools import chain 
workbook = xlsxwriter.Workbook('/content/TestCaseOutput.xlsx')
worksheet = workbook.add_worksheet()

# Read the file
df1 = pd.read_csv(r'/content/TestData.csv')
df = pd.DataFrame(df1, columns =['AccpCriteria'])
 # Iterate over the sequence of column names
for column in reversed(df.columns):
    
    # Select column contents by column
    # name using [] operator
    columnSeriesObj = df[column]
    print('Column Name : ', column)
    print('Column Contents : ', columnSeriesObj.values)

    Stringval = str(columnSeriesObj.values)
    print(Stringval)

from pipelines import pipeline
nlp = pipeline("question-generation") 

nlp(Stringval)


# Driver code
word = nlp(Stringval)
#word1 = split(word)
#returns only values

values = []
for dictionary in word:
     values.extend([v for k, v in dictionary.items() if k == 'answer'])
print(values)

def split(word):
    return [char for char in word]
word2 = split(values)
header1 = "Steps"
header2 = "Expected_result"
worksheet.write('A1', header1)
worksheet.write('B1', header2)

row = 1
column = 1
content = word2
  
# iterating through content list
for item in content :
    worksheet.write(row, column, item)
    row += 1

values1 = []
for dictionary in word:
     values1.extend([v for k, v in dictionary.items() if k == 'question'])
word2 = split(values1)

n = ["happens","do", "will", "is", "are", "if", "?", "[", "]", "``", "'", "'", "[ "]     
stopwords = set(n)
word_tokens = word_tokenize(str(values1))

new_strings = [w for w in word_tokens if not w.lower() in stopwords]

new_strings = []
for string in word_tokens:
  if string not in stopwords:
     
     #Prefix = "Check" 
     new_string = string.replace("What", "Check whether")
     new_strings.append(new_string)
     #listToStr = split(new_strings)
     listToStr = ' '.join(new_strings)
     
     listToStr1 = listToStr.split(', ')


print(listToStr1)

row = 1
column = 0
word3 = listToStr1

# iterating through content list
for item in word3:
    worksheet.write_string(row, column, item)
    row += 1
workbook.close()   




/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Cloning into 'question_generation'...
remote: Enumerating objects: 265, done.
remote: Counting objects: 100% (109/109), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 265 (delta 100), reused 95 (delta 95), pack-reused 156
Receiving objects: 100% (265/265), 289.06 KiB | 17.00 MiB/s, done.
Resolving deltas: 100% (146/146), done.
/content/question_generation/question_generation/question_generation/question_generation/question_generation/question_generation/question_generation/question_generation/question_generation/question_generation/question_generation/question_generation/question_generation/question_generation/question_gene

In [ ]:
!pip install transformers
!pip install timeline 
!pip install pipeline
!pip install pipelines 
!pip install openpyxl 
!pip install xlrd
!pip install xlsxwriter 

     |████████████████████████████████| 2.8 MB 5.3 MB/s 
     |████████████████████████████████| 3.3 MB 67.7 MB/s 
     |████████████████████████████████| 52 kB 1.3 MB/s 
     |████████████████████████████████| 895 kB 47.9 MB/s 
     |████████████████████████████████| 636 kB 64.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Created wheel for timeline: filename=timeline-0.0.7-py3-none-any.whl size=17005 sha256=746d8177f51b400e8ee3ed92d2930afc7c7eade1e441172ed767bffae839a4b9
  Stored in directory: /root/.cache/pip/wheels/eb/03/7d/af6f2e055c7a95e6e9dcd5e64a9738678cac6bd8b05d4e1bd3
Successfully built timeline
     |████████████████████████████████| 161 kB 6.5 MB/s 
     |████████████████████████████████| 161 kB 45.4 MB/s 
     |████████████████████████████████| 161 kB 49.2 MB/s 
     |████████████████████████████████| 159 kB 44.0 MB/s 
     |████████████████████████████████| 159 

In [2]:
#Actual Working code

!pip install -U transformers==3.0.0 --quiet
!python -m nltk.downloader punkt

!git clone https://github.com/patil-suraj/question_generation.git
%cd question_generation
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import word_tokenize, sent_tokenize
from keras.preprocessing.text import Tokenizer

from transformers import pipeline
import timeline
import pandas as pd
import csv 
import openpyxl
import xlsxwriter
from itertools import chain 
workbook = xlsxwriter.Workbook('/content/TestCaseOutput.xlsx')
worksheet = workbook.add_worksheet()

# Read the file
df1 = pd.read_csv(r'/content/TestScenario1.csv', error_bad_lines=False)
df = pd.DataFrame(df1, columns =['AccpCriteria'])
for column in reversed(df.columns):
    

    columnSeriesObj = df[column]
    print('Column Name : ', column)
    print('Column Contents : ', columnSeriesObj.values)

    Stringval = str(columnSeriesObj.values)
    print(Stringval)

from pipelines import pipeline
nlp = pipeline("question-generation") 

nlp(Stringval)
word = nlp(Stringval)

values = []
for dictionary in word:
     values.extend([v for k, v in dictionary.items() if k == 'answer'])
print(values)

def split(word):
    return [char for char in word]
word2 = split(values)
header1 = "Steps"
header2 = "Expected_result"
worksheet.write('A1', header1)
worksheet.write('B1', header2)

row = 1
column = 1
content = word2
  
# iterating through content list
for item in content :
    worksheet.write(row, column, item)
    row += 1

values1 = []
for dictionary in word:
     values1.extend([v for k, v in dictionary.items() if k == 'question'])
word2 = split(values1)

n = ["happens","do", "will", "is", "are", "if", "?", "[", "]", "``", "'", "'", "[ "]     
stopwords = set(n)
word_tokens = word_tokenize(str(values1))

new_strings = [w for w in word_tokens if not w.lower() in stopwords]

new_strings = []
for string in word_tokens:
  if string not in stopwords:
     
     #Prefix = "Check" 
     #new_string = string.replace("What", "Check whether")
     new_strings.append(string)
     listToStr = ' '.join(new_strings)
     listToStr1 = listToStr.split(', ')
      
append_str = 'Check if '
pre_res = [append_str + sub for sub in listToStr1]
#suf_res = [sub + append_str for sub in values]
  
print("list after prefix addition : " + str(pre_res))

print(pre_res)

row = 1
column = 0
word3 = pre_res

# iterating through content list
for item in word3:
    worksheet.write_string(row, column, item)
    row += 1
workbook.close()   




     |████████████████████████████████| 754 kB 5.4 MB/s 
     |████████████████████████████████| 1.2 MB 42.1 MB/s 
     |████████████████████████████████| 3.0 MB 39.9 MB/s 
/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Cloning into 'question_generation'...
remote: Enumerating objects: 265, done.
remote: Counting objects: 100% (109/109), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 265 (delta 100), reused 95 (delta 95), pack-reused 156
Receiving objects: 100% (265/265), 289.06 KiB | 3.61 MiB/s, done.
Resolving deltas: 100% (146/146), done.
/content/question_generation
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.

Downloading:   0%|          | 0.00/627 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


['PWTIndus', 'DID F188']
list after prefix addition : ["Check if 'What I select in Appli Signature step having Partner ! = Nissan and Indus category ! = BMS ", "Check if 'What field must be automatically filled in with the value contained in the field Software Virtual file"]
["Check if 'What I select in Appli Signature step having Partner ! = Nissan and Indus category ! = BMS ", "Check if 'What field must be automatically filled in with the value contained in the field Software Virtual file"]


In [ ]:
import timeline
from transformers import pipeline
nlp = pipeline(task='text-generation', model='gpt2')
nlp("My name is Fernando, I am from Mexico and", max_length=30, num_return_sequences=5)

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/230 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/modeling_auto.py:791: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


[{'generated_text': 'My name is Fernando, I am from Mexico and I am in this country to be the best player I have ever had the pleasure and support to play'},
 {'generated_text': 'My name is Fernando, I am from Mexico and I lived here for eight years. And now it is back to Honduras, and I am getting a'},
 {'generated_text': 'My name is Fernando, I am from Mexico and I work in food justice.\n\nHoney does the best she can but she does what she'},
 {'generated_text': 'My name is Fernando, I am from Mexico and have three younger brothers, Luis and Enrique. But I love to help my family and if you haven'},
 {'generated_text': 'My name is Fernando, I am from Mexico and a very young boy."\n\nHe also claimed that his father was involved in an affair with his'}]

In [ ]:
 !pip install -U transformers==3.0.0 --quiet
!python -m nltk.downloader punkt

!git clone https://github.com/patil-suraj/question_generation.git
%cd question_generation
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import word_tokenize, sent_tokenize
from keras.preprocessing.text import Tokenizer

from transformers import pipeline
import timeline
import pandas as pd
import csv 
import openpyxl
import xlsxwriter
import timeline


from itertools import chain 
workbook = xlsxwriter.Workbook('/content/TestCaseOutput.xlsx')
worksheet = workbook.add_worksheet()

# Read the file
df1 = pd.read_csv(r'/content/TestScenario6.csv')
df = pd.DataFrame(df1, columns =['AccpCriteria'])
 for column in reversed(df.columns):
    

    columnSeriesObj = df[column]
    print('Column Name : ', column)
    print('Column Contents : ', columnSeriesObj.values)

    Stringval = str(columnSeriesObj.values)
    print(Stringval)

 from pipelines import pipeline
nlp = pipeline("question-generation")

nlp(Stringval)


# Driver code
word = nlp(Stringval)
#word1 = split(word)
#returns only values

values = []
for dictionary in word:
     values.extend([v for k, v in dictionary.items() if k == 'answer'])
print(values)

from transformers import pipeline
nlp = pipeline(task='text-generation', model='gpt2')
nlp(values, max_length=20, num_return_sequences=1)


/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Cloning into 'question_generation'...
remote: Enumerating objects: 265, done.
remote: Counting objects: 100% (109/109), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 265 (delta 100), reused 95 (delta 95), pack-reused 156
Receiving objects: 100% (265/265), 289.06 KiB | 3.57 MiB/s, done.
Resolving deltas: 100% (146/146), done.
/content/question_generation/question_generation/question_generation/question_generation/question_generation/question_generation/question_generation
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Column Name :  AccpCriteria
Column 

/usr/local/lib/python3.7/dist-packages/transformers/modeling_auto.py:791: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 502

[[{'generated_text': 'x02xx_PLT_Users group, but because the new ID would be different I was'}],
 [{'generated_text': 'tries to promote the ECU Plant item in your home. A complete list and instructions are available'}],
 [{'generated_text': 'an error-No promotion available is diplayed by the end of the program.\n\nPlease see'}],
 [{'generated_text': 'Inwork state workers do not have jobs to do, and they must earn minimum wage and hours and'}],
 [{'generated_text': 'x02xx_PLT_Users group,\n\n#1650;\n\n/* *'}],
 [{'generated_text': "'WHEN AS user tries to promote the ECU Plant item, she will stop by the Plant"}],
 [{'generated_text': 'an error-No promotion available is diplayed here. There is an additional link for additional updates.'}],
 [{'generated_text': "Inwork state that as soon as the state's health department began conducting its investigation, it notified the"}],
 [{'generated_text': 'x02xx_PLT_Users group (C:\\WINDOWS\\SYSTEM32\\dvd'}],
 [{'generated_text': 'Diaglandi, of cour

In [ ]:
!pip install -U transformers==3.0.0 --quiet
!python -m nltk.downloader punkt

!git clone https://github.com/patil-suraj/question_generation.git
%cd question_generation
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import word_tokenize, sent_tokenize
from keras.preprocessing.text import Tokenizer

#from transformers import pipeline

#import timeline
import transformers
import pandas as pd
import csv 
import openpyxl
import xlsxwriter
from itertools import chain 
workbook = xlsxwriter.Workbook('/content/TestCaseOutput.xlsx')
worksheet = workbook.add_worksheet()

# Read the file
df1 = pd.read_csv(r'/content/TestScenario1.csv', error_bad_lines=False)
df = pd.DataFrame(df1, columns =['AccpCriteria'])
for column in reversed(df.columns):
    

    columnSeriesObj = df[column]
    print('Column Name : ', column)
    print('Column Contents : ', columnSeriesObj.values)

    Stringval = str(columnSeriesObj.values)
    print(Stringval)
#Text summarization
from transformers import pipeline
#txt = Stringval
summarizer = pipeline("summarization")
for summary1 in columnSeriesObj.values:
 summarized = summarizer(summary1, max_length=20, min_length=15)
 print(summarized)
 summ=' '.join([str(i) for i in summarized])
 summ=summ.replace("{","")
 summ=summ.replace("''","")
 summ=summ.replace("''","")
 summ1=len(summ.split())
 print(summ1)

 from pipelines import pipeline
nlp = pipeline("question-generation") 
val1 = []
for dictionary in summ1:
     val1.extend([v for k, v in dictionary.items() if k == 'summary_text'])
print(val1)
nlp(values1)
word = nlp(val1)

values = []
for dictionary in word:
     values.extend([v for k, v in dictionary.items() if k == 'answer'])
print(values)

def split(word):
    return [char for char in word]
word2 = split(values)
header1 = "Steps"
header2 = "Expected_result"
worksheet.write('A1', header1)
worksheet.write('B1', header2)

row = 1
column = 1
content = word2
  
# iterating through content list
for item in content :
    worksheet.write(row, column, item)
    row += 1

values1 = []
for dictionary in word:
     values1.extend([v for k, v in dictionary.items() if k == 'question'])
word2 = split(values1)

n = ["happens","do", "will", "is", "are", "if", "?", "[", "]", "``", "'", "'", "[ "]     
stopwords = set(n)
word_tokens = word_tokenize(str(values1))

new_strings = [w for w in word_tokens if not w.lower() in stopwords]

new_strings = []
for string in word_tokens:
  if string not in stopwords:
     
     #Prefix = "Check" 
     #new_string = string.replace("What", "Check whether")
     new_strings.append(string)
     listToStr = ' '.join(new_strings)
     listToStr1 = listToStr.split(', ')
      
append_str = 'Check if '
pre_res = [append_str + sub for sub in listToStr1]
#suf_res = [sub + append_str for sub in values]
  
print("list after prefix addition : " + str(pre_res))

print(pre_res)

row = 1
column = 0
word3 = pre_res

# iterating through content list
for item in word3:
    worksheet.write_string(row, column, item)
    row += 1
workbook.close() 

/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
fatal: destination path 'question_generation' already exists and is not an empty directory.
/content/question_generation
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Column Name :  AccpCriteria
Column Contents :  ['GIVEN THAT I select a PWTIndus in In Appli Signature step having Partner != Nissan and Indus category != BMS.'
 'WHEN I click on Approve I fill in all mandatory fields and I click on OK.'
 'THEN in the Tab Delivery Letter the field DID F188 must be automatically filled in with the value contained in the field Software Virtual file.']
['GIVEN THAT I select a PWTI

In [13]:
!pip install transformers
!pip install timeline
!pip install pipeline
from transformers import pipeline
#from transformers import timeline
nlp = pipeline(task='text-generation', model='gpt2')
nlp("My name is Fernando, I am from Mexico and", max_length=30, num_return_sequences=5)
#String1 = nlp(values, max_length=10, num_return_sequences=1)

/usr/local/lib/python3.7/dist-packages/transformers/modeling_auto.py:791: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


[{'generated_text': 'My name is Fernando, I am from Mexico and it started my whole career there. I love having a kid. Our little brother Angel is 6 now'},
 {'generated_text': 'My name is Fernando, I am from Mexico and I\'m married to an African-American black man." The "black name" is often used to'},
 {'generated_text': 'My name is Fernando, I am from Mexico and I come from my family." She said she has never been to a school where a single black student'},
 {'generated_text': 'My name is Fernando, I am from Mexico and I was born here. My father is from Honduras and his grandfather is from Puerto Rico. My mother'},
 {'generated_text': 'My name is Fernando, I am from Mexico and the year was 1994. The first one I heard about was the one after they came out with a'}]

In [22]:

%%capture
!pip install wordwise
!git clone https://github.com/jaketae/wordwise.git
from wordwise import Extractor

extractor = Extractor()
text = """
         Supervised learning is the machine learning task of 
         learning a function that maps an input to an output based 
         on example input-output pairs.[1] It infers a function 
         from labeled training data consisting of a set of 
         training examples.[2] In supervised learning, each 
         example is a pair consisting of an input object 
         (typically a vector) and a desired output value (also 
         called the supervisory signal). A supervised learning 
         algorithm analyzes the training data and produces an 
         inferred function, which can be used for mapping new 
         examples. An optimal scenario will allow for the algorithm 
         to correctly determine the class labels for unseen 
         instances. This requires the learning algorithm to  
         generalize from the training data to unseen situations 
         in a 'reasonable' way (see inductive bias).
      """
keywords = extractor.generate(text, 3)
print(keywords)

OSError: ignored

In [25]:
%%capture
!pip install transformers
from transformers import pipeline

tags = [ "robots", "space & cosmos", "scientific discovery", "microbiology", "archeology"]
text = """Europa is one of Jupiter’s four large Galilean moons. 
          And in a paper published Monday in the Astrophysical Journal, 
          Dr. Batygin and a co-author, Alessandro Morbidelli, 
          a planetary scientist at the Côte d’Azur Observatory in France, 
          present a theory explaining how some moons form around gas giants like 
          Jupiter and Saturn, suggesting that millimeter-sized grains of hail 
          produced during the solar system’s formation became trapped 
          around these massive worlds, taking shape one at a time into 
          the potentially habitable moons we know today."""

classifier = pipeline("zero-shot-classification")
outputs = classifier(text, tags, multi_class=True)
for label, score in zip(outputs["labels"], outputs["scores"]):
    print(f"{label}: {score:.3f}")

KeyError: ignored